In [ ]:
%%bash
pip install regex requests

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from torch.utils.data import Dataset
import numpy as np 
import torch
import os
import json
import pickle
import random

def pad_tensor(vec, pad, dim):
    pad_size = list(vec.shape)
    pad_size[dim] = pad - vec.size(dim)
    return torch.cat([vec, torch.zeros(*pad_size)], dim=dim)	

use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')

#roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
#roberta.eval()

In [ ]:
!pip3 install py-rouge
import rouge

def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)


     |████████████████████████████████| 61kB 4.8MB/s 


In [3]:
import pickle
cnn_dataset_path = '/content/drive/My Drive/Colab Notebooks/summarization_data/cnn_dataset_test.pkl'
dm_dataset_path = '/content/drive/My Drive/Colab Notebooks/summarization_data/dailymail_dataset_test.pkl'
file = open(cnn_dataset_path, "rb")

stories_cnn = pickle.load(file)
print('Loaded Stories %d' % len(stories_cnn))

file = open(dm_dataset_path, "rb")

stories_dm = pickle.load(file)
print('Loaded Stories %d' % len(stories_dm))

Loaded Stories 186
Loaded Stories 436


In [4]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [5]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def preprocess(text):
    #text = text.lower() # lowercase
    text = text.split() # convert have'nt -> have not
    for i in range(len(text)):
        word = text[i]
        if word in contraction_mapping:
            text[i] = contraction_mapping[word]
    text = " ".join(text)
    #text = text.split()
    '''newtext = []
    for word in text:
        if word not in stop_words:
            newtext.append(word)
    text = " ".join(newtext)'''
    #text = text.replace("'s",'') # convert your's -> your
    text = re.sub(r'\(.*\)','',text) # remove (words)
    #text = re.sub(r'[^a-zA-Z0-9. ]','',text) # remove punctuations
    #text = re.sub(r'\.',' . ',text)
    text = text.replace("-", '')
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
import nltk.data
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

text = []
summary = []
for story in stories_cnn:
  x = ". ".join(story['highlights'])
  x += '.'
  summary.append(x)
  x = preprocess(story['story'])
  x = tokenizer.tokenize(x)
  text.append(x)
for story in stories_dm:
  x = ". ".join(story['highlights'])
  x += '.'
  summary.append(x)
  x = preprocess(story['story'])
  x = tokenizer.tokenize(x)
  text.append(x)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
print(len(text))

622


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class SelfAttention(nn.Module):
	def __init__(self, batch_size=32, hidden_size=200, embedding_length=768):
		super(SelfAttention, self).__init__()

		"""
		Arguments
		---------
		batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
		output_size : 2 = (pos, neg)
		hidden_sie : Size of the hidden_state of the LSTM
		vocab_size : Size of the vocabulary containing unique words
		embedding_length : Embeddding dimension of GloVe word embeddings
		weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
		
		--------
		
		"""

		self.batch_size = batch_size
		self.hidden_size = hidden_size

		self.bilstm = nn.LSTM(embedding_length, hidden_size, 2, dropout=0.2, bidirectional=True)
		# We will use da = 350, r = 30 & penalization_coeff = 1 as per given in the self-attention original ICLR paper
		self.W_s1 = nn.Linear(2*hidden_size, 350)
		self.W_s2 = nn.Linear(350, 30)
		self.fc_layer = nn.Linear(30*2*hidden_size, 2000)

	def attention_net(self, lstm_output):

		"""
		Now we will use self attention mechanism to produce a matrix embedding of the input sentence in which every row represents an
		encoding of the inout sentence but giving an attention to a specific part of the sentence. We will use 30 such embedding of 
		the input sentence and then finally we will concatenate all the 30 sentence embedding vectors and connect it to a fully 
		connected layer of size 2000 which will be connected to the output layer of size 2 returning logits for our two classes i.e., 
		pos & neg.
		Arguments
		---------
		lstm_output = A tensor containing hidden states corresponding to each time step of the LSTM network.
		---------
		Returns : Final Attention weight matrix for all the 30 different sentence embedding in which each of 30 embeddings give
				  attention to different parts of the input sentence.
		Tensor size : lstm_output.size() = (batch_size, num_seq, 2*hidden_size)
					  attn_weight_matrix.size() = (batch_size, 30, num_seq)
		"""
		attn_weight_matrix = self.W_s2(F.tanh(self.W_s1(lstm_output)))
		attn_weight_matrix = attn_weight_matrix.permute(0, 2, 1)
		attn_weight_matrix = F.softmax(attn_weight_matrix, dim=2)

		return attn_weight_matrix

	def forward(self, input, batch_size=None):

		""" 
		Parameters
		----------
		input_sentence: input_sentence of shape = (batch_size, num_sequences)
		batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
		
		Returns
		-------
		Output of the linear layer containing logits for pos & neg class.
		
		"""

		#input = self.word_embeddings(input_sentences)
		input = input.permute(1, 0, 2)
		if batch_size is None:
			h_0 = Variable(torch.zeros(4, self.batch_size, self.hidden_size).cuda())
			c_0 = Variable(torch.zeros(4, self.batch_size, self.hidden_size).cuda())
		else:
			h_0 = Variable(torch.zeros(4, batch_size, self.hidden_size).cuda())
			c_0 = Variable(torch.zeros(4, batch_size, self.hidden_size).cuda())

		output, (h_n, c_n) = self.bilstm(input, (h_0, c_0))
		output = output.permute(1, 0, 2)
		# output.size() = (batch_size, num_seq, 2*hidden_size)
		# h_n.size() = (1, batch_size, hidden_size)
		# c_n.size() = (1, batch_size, hidden_size)
		attn_weight_matrix = self.attention_net(output)
		# attn_weight_matrix.size() = (batch_size, r, num_seq)
		# output.size() = (batch_size, num_seq, 2*hidden_size)
		hidden_matrix = torch.bmm(attn_weight_matrix, output)
		# hidden_matrix.size() = (batch_size, r, 2*hidden_size)
		# Let's now concatenate the hidden_matrix and connect it to the fully connected layer.
		fc_out = self.fc_layer(hidden_matrix.view(-1, hidden_matrix.size()[1]*hidden_matrix.size()[2]))
		# logits.size() = (batch_size, output_size)

		return fc_out

class LSTM1(nn.Module):
	def __init__(self, batch_size=256, hidden_size=200, embedding_length=768, num_labels=3):
		super(LSTM1, self).__init__()

		self.premise_net = SelfAttention(batch_size, hidden_size, embedding_length)
		self.hypothesis_net = SelfAttention(batch_size, hidden_size, embedding_length)
		self.fc = nn.Linear(4000, 3)

	def forward(self, premise, hypothesis):
		rep1 = self.premise_net(premise)
		rep2 = self.hypothesis_net(hypothesis)
		rep = torch.cat([rep1, rep2], 1)
		return self.fc(rep)

In [9]:
!pip3 install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 4.7MB/s 


In [10]:
from pytorch_pretrained_bert import BertTokenizer, BertModel
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model = bert_model.cuda()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
lstm = LSTM1(batch_size = 1).to(device)
lstm.load_state_dict(torch.load("/content/drive/My Drive/IG wala/checkpoints/networkbertlstm1_train_epoch_27.ckpt"))
    
def pad_tensor(vec, pad, dim):
    pad_size = list(vec.shape)
    pad_size[dim] = pad - vec.size(dim)
    return torch.cat([vec, torch.zeros(*pad_size)], dim=dim)	

def predict(premise, hypothesis):
    marked_text = "[CLS] " + premise + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    bert_model.eval()
    with torch.no_grad():
      encoded_layers, _ = bert_model(tokens_tensor.to(device), segments_tensors.to(device))
      embedding = encoded_layers[11][0]
      premise = np.array(embedding.cpu())
    marked_text = "[CLS] " + hypothesis + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    bert_model.eval()
    with torch.no_grad():
      encoded_layers, _ = bert_model(tokens_tensor.to(device), segments_tensors.to(device))
      embedding = encoded_layers[11][0]
      hypothesis = np.array(embedding.cpu())
    premise = torch.tensor(premise).type(torch.FloatTensor)
    hypothesis = torch.tensor(hypothesis).type(torch.FloatTensor)
    premise = pad_tensor(premise, 250, 0)
    hypothesis = pad_tensor(hypothesis, 250, 0)
    premise = premise.unsqueeze(0)
    hypothesis = hypothesis.unsqueeze(0)
    y_pred = lstm(premise.to(device), hypothesis.to(device))
    return y_pred


100%|██████████| 231508/231508 [00:00<00:00, 893727.19B/s]


In [43]:
lab = {
				"neutral" : 0,
				"contradiction" : 1,
				"entailment" : 2
		}
graph = {}
sentences = text[131]
print(sentences)
for sentence1 in sentences:
   adj_list = []
   for sentence2 in sentences:
     if sentence1 == sentence2:
       continue
     s = []
     s.append(sentence1)
     s.append(sentence2)
     if len(s) != 2:
       continue
     try:
      with torch.no_grad():
         prediction = predict(s[0], s[1])
         _, prediction = prediction.max(1)
         #print(prediction)
         if prediction == 2:
          #print("yes")
          adj_list.append(sentence2)
     except Exception as e:
       print(e)
     if len(adj_list) > 0:
       graph[sentence1] = adj_list
#print(graph)
result = []
while len(graph)>0:
  s1 = next(iter(graph))
  if(len(s1)<=5):
    del graph[s1]
    continue
  result.append(s1)
  f = 0
  i = 0
  s2 = graph[s1][0]
  while(len(s2)<=5):
    try:
      del graph[s2]
    except Exception as e:
      print(e)
    i = i+1
    try:
      s2 = graph[s1][i]
    except Exception as e:
      print(e)
      f = 1
      break
  if f == 0:
    result.append(s2)
  del graph[s1]
  try:
    del graph[s2]
  except Exception as e:
    print(e)
#print(result)
result = list(dict.fromkeys(result))
#print(result)
result = [x[:len(x)-1] + "." for x in result]
result = " ".join(result)
print(result)
#print(summary[0])

['  It was not the present cricket fans would have asked for when they woke up bleary eyed on Christmas morning.', 'Jacques Kallis, arguably the greatest allrounder in the history of the game, has announced he will retire from Test match cricket following the second Test against India which begins on December 26.', 'Kallis, 38, made the announcement in a statement Wednesday following a career which has spanned 18 years and 165 Test matches.', '"It has been an honor and a privilege to have been part of the South African Test team since making my debut 18 years ago," he said.', '"I have enjoyed every moment out in the middle but I just feel that the time is right to hang up my Test whites.', '"It was not an easy decision to come to, especially with Australia around the corner and the success this team is enjoying, but I feel that I have made my contribution in this format."', 'Kallis made his debut against England as a 20yearold in December 1995 and has scored 13,174 runs at an average o

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


'"I\'m not sure we will ever see another player of that stature very soon.'
'"I am fortunate enough to have ended my Test career amongst a group of talented cricketers but more importantly, friends whom I will cherish for years to come.'
'"I am fortunate enough to have ended my Test career amongst a group of talented cricketers but more importantly, friends whom I will cherish for years to come.'
'"It was not an easy decision to come to, especially with Australia around the corner and the success this team is enjoying, but I feel that I have made my contribution in this format."'
'"It was not an easy decision to come to, especially with Australia around the corner and the success this team is enjoying, but I feel that I have made my contribution in this format."'
'"It was not an easy decision to come to, especially with Australia around the corner and the success this team is enjoying, but I feel that I have made my contribution in this format."'
'"I am fortunate enough to have ended m

In [96]:
graph = {}
sentences = text[164]
print(sentences)
for sentence1 in sentences:
   adj_list = []
   for sentence2 in sentences:
     if sentence1 == sentence2:
       continue
     s = []
     s.append(sentence1)
     s.append(sentence2)
     if len(s) != 2:
       continue
     try:
      with torch.no_grad():
         prediction = predict(s[0], s[1])
         _, prediction = prediction.max(1)
         #print(prediction)
         if prediction == 2:
          #print("yes")
          adj_list.append(sentence2)
     except Exception as e:
       print(e)
     if len(adj_list) > 0:
       graph[sentence1] = adj_list
graph_list = sort_by_values_len(graph)
result = []
if len(graph)>0 :
  h = 0
  result.append(list(graph_list[h].keys())[0])
  while(dominating_set(result, sentences, graph) == 0):
    h = h + 1
    result.append(list(graph_list[h].keys())[0])
result = [x[:len(x)-1] + "." for x in result]
result = "".join(result)
print(result)


['Washington   Senate Republicans will block Democrats from taking up a White Housebacked bill to cap rates on some student loans unless Democrats allow a vote on a GOP alternative, the secondranking Senate Republican said Monday.', 'Arizona Sen. Jon Kyl predicted such a confrontation could lead to an ultimate compromise on the legislation before rates are set to double in July, something lawmakers from both parties and the White House want to prevent.', '"We\'ll defeat cloture," Kyl said, using the legislative parlance for a key procedural vote scheduled for Tuesday that requires 60 votes to succeed.', 'If Republicans prove Democrats cannot move a bill without GOP support, "I presume leaders in the House and Senate will get together and find a way to ensure the interest rate does not double," Kyl said.', 'An aide to Senate Majority Leader Harry Reid, DNevada, said negotiations over the issue are still under way.', 'Interest rates on federal student loans are scheduled to double from t

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Arizona Sen. Jon Kyl predicted such a confrontation could lead to an ultimate compromise on the legislation before rates are set to double in July, something lawmakers from both parties and the White House want to prevent.


In [64]:
print(summary[141])

Manchester City move to top of English Premier League with 5-0 win over Aston Villa. Rivals Manchester United drop to second after losing 1-0 to Norwich City. Third-placed Chelsea suffer 2-1 defeat away to surprise team West Brom. Arsenal win 5-2 in London derby with Tottenham, who have a player sent off.


In [97]:
text1 = " ".join(sentences)
print(text1)

"We'll defeat cloture," Kyl said, using the legislative parlance for a key procedural vote scheduled for Tuesday that requires 60 votes to succeed. An aide to Senate Majority Leader Harry Reid, DNevada, said negotiations over the issue are still under way. Arizona Sen. Jon Kyl predicted such a confrontation could lead to an ultimate compromise on the legislation before rates are set to double in July, something lawmakers from both parties and the White House want to prevent. But Democrats and Republicans disagree over how to offset the $6 billion it would cost to keep the rates down: Democrats want to eliminate a tax break for some corporations while Republicans want to pull funds from a preventative health care fund set up under the health reform law. Each spending plan is opposed by the other party. If Republicans prove Democrats cannot move a bill without GOP support, "I presume leaders in the House and Senate will get together and find a way to ensure the interest rate does not dou

["Vatican City   Pope Benedict XVI delivered his final public prayer ceremony Sunday to a crowd of thousands at St. Peter's Square in Vatican City, amid rumors of a sex scandal brewing as he prepares to step down.", '"Thank you for your affection," the pope told the crowd as he appeared at the window of his apartment overlooking the square.', 'As is customary, he opened the weekly Angelus prayer with a short sermon.', "Benedict spoke on the Transfiguration of Christ, one of the key moments in Jesus' life on Earth, when, according to the church, he took three disciples to pray on a mountain.", 'During his prayers, Jesus was miraculously changed and filled with light.', "Read the full text of Benedict's sermon The crowd interrupted Benedict with rousing applause, as he told them that God wanted him to do the same.", '"The Lord is calling me to go on top of the hill, to dedicate myself once more to prayer and meditation," he said.', '"But this does not mean to abandon the church."', 'Resignation and rumors This month, Benedict shocked the world with the announcement that he would step down.', 'Thursday is his last day as acting pope.', "The Italian media has been abuzz in recent days with allegations that gay clergy may have made themselves vulnerable to blackmail by male prostitutes, setting off speculation that a brewing scandal may have triggered Benedict's resignation.", 'The Vatican vehemently denied the allegations Saturday.', 'Interactive: Where does the pope live?', 'Vatican Secretary of State Tarcisio Bertone said it was "deplorable" that as the time for the Roman Catholic cardinals to elect a new pope approaches, a rash of "often unverified, unverifiable or completely false news stories" has appeared.', 'As the first pontiff to step down in six centuries, Benedict said he is resigning "because of advanced age."', 'He is 85.', '"Strength of mind and body are necessary, strength which in the last few months has deteriorated in me to the extent that I have had to recognize my incapacity to adequately fulfill the ministry entrusted to me," Benedict said when he announced his retirement this month.', "Shortly after the pope's announcement nearly two weeks ago, a different rumor began making its way through the Italian media: Allegations that the pontiff's decision was linked to a medical procedure to replace the battery in his pacemaker.", 'A Vatican spokesman dismissed the idea at the time.', 'CNN Senior Vatican Analyst John Allen has said that clergy members trying to cover up their sex lives may well be creating problems in the Catholic Church, as they have in the past.', 'But he believes that the current unsourced speculation in the Italian media should be taken with a grain of salt.', 'Several years ago, Benedict suggested he would be open to resigning should his health fail, Allen has said.', 'Wrapping up a papacy After the sermon, he chanted the Angelus, a common Catholic prayer, with the crowd.', 'It begins with: "The angel of the Lord declared unto Mary.', 'And she conceived by the power of Holy Spirit.', 'Hail Mary, full of grace, the Lord is with you.', 'Blessed are you among women, and blessed is the fruit of your womb, Jesus."', 'The throng of spectators extended beyond the borders of the piazza, some waving flags from nations around the world and holding up signs with words of thanks for the departing head of the Catholic Church.', 'He ended his message with: "Let\'s always be close in prayer."', "Sunday's Angelus will not be Benedict's final public appearance, as he is scheduled to hold a general audience Wednesday, according to the Vatican.", 'He will hold a "meet and greet" with cardinals late Thursday morning before stepping down the same evening, the Vatican website said.', 'Benedict was dean of the College of Cardinals in 2005 when he became the sixth German to be picked as pope, albeit the first since the 11th century.', "At the time, the church was facing several pressing issues, including a growing crisis over the church's role in handling molestation accusations against priests around the world.", 'The pontiff will leave office at 8 p.m. on February 28, twoandahalf weeks after he shocked the Roman Catholic world by announcing his resignation.', "The Vatican has said a new pope will be in place for the church's Easter celebrations on March 31.", 'Benedict will begin his retirement in the papal summer residence in Castel Gandolfo before he withdraws to a monastery being built for him in Vatican City, a Vatican spokesman has said.', 'He will then lead a cloistered life, the spokesman said.', "CNN's Ben Wedeman reported from Vatican City, and Ben Brumfield reported and wrote from Atlanta.", "CNN's Hada Messia contributed to this report."]



The Italian media has been abuzz in recent days with allegations that gay clergy may have made themselves vulnerable to blackmail by male prostitutes, setting off speculation that a brewing scandal may have triggered Benedict's resignation.

In [ ]:
gen_summary = []
for it in range(100, 101):
  print('\r', it, end = '')
  graph = {}
  sentences = text[it]
  for sentence1 in sentences:
   adj_list = []
   for sentence2 in sentences:
     if sentence1 == sentence2:
       continue
     s = []
     s.append(sentence1)
     s.append(sentence2)
     if len(s) != 2:
       continue
     try:
      with torch.no_grad():
         prediction = predict(s[0], s[1])
         _, prediction = prediction.max(1)
         #print(prediction)
         if prediction == 2:
          adj_list.append(sentence2)
     except Exception as e:
       print(e)
     if len(adj_list) > 0:
       graph[sentence1] = adj_list
  result = []
  print(graph)
  while len(graph)>0:
    s1 = next(iter(graph))
    if(len(s1)<=5):
      del graph[s1]
      continue
    result.append(s1)
    f = 0
    i = 0
    s2 = graph[s1][0]
    while(len(s2)<=5):
      try:
        del graph[s2]
      except:
        pass
      i = i+1
      try:
        s2 = graph[s1][i]
      except:
        f = 1
        break
    if f == 0:
      result.append(s2)
    del graph[s1]
    try:
      del graph[s2]
    except:
      pass
  result = list(dict.fromkeys(result))
  result = [x[:len(x)-1] + "." for x in result]
  result = " ".join(result)
  gen_summary.append(result)
  if it == 3:
    print(result)

#file = open('/content/drive/My Drive/Colab Notebooks/pickle/gen_summaries_cnndm_3.pkl','wb+')
#pickle.dump((gen_summary), file)
#file.close()

 100

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


{}


In [ ]:
file = open('/content/drive/My Drive/Colab Notebooks/pickle/gen_summaries_cnndm_1.pkl','rb+')
sublist1 = pickle.load(file)
print(len(sublist1))
file.close()

100


In [ ]:
print(sublist[23])
print(sublist1[23])

By Daily Mail Reporter PUBLISHED: 13:43 EST, 29 October 2013 | UPDATED: 17:42 EST, 29 October 2013 A Florida mother has been sentenced to 54 years in federal prison after pleading guilty to sexually abusing her children together with her husband, a decorated Marine. He was awarded the Purple Heart following a tour of Afghanistan. Doomed: Sarah Adleta, 29, was sentenced to 54 years in federal prison for abusing her two children.
  One senator who has survived the vice presidential vetting process described it this way: Like having a colonoscopy without anesthesia. Susana Martinez, Louisiana Gov. The probing, highly personal examination that a presidential candidate uses to pick a running mate is uniquely American. Rob Portman, Florida Sen. Marco Rubio, New Hampshire Sen. Kelly Ayotte, New Jersey Gov. Yet it is entirely shielded from public view: Those who submit themselves to it must bow to a secret, often uncomfortable "deep dive" for information by a small, exclusive tribunal of lawye

In [ ]:
!pip install py-rouge
import rouge

def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

for aggregator in ['Avg', 'Best']:
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)


    
    all_hypothesis = [summary[0], summary[1]]
    all_references = [gen_summary[0], gen_summary[1]]

    scores = evaluator.get_scores(all_hypothesis, all_references)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
            for hypothesis_id, results_per_ref in enumerate(results):
                nb_references = len(results_per_ref['p'])
                for reference_id in range(nb_references):
                    print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                    print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
            print()
        else:
            print(prepare_results(results['p'], results['r'], results['f']))
    print()

Evaluation with Avg
	rouge-1:	P: 60.61	R: 27.09	F1: 37.43
	rouge-2:	P: 30.70	R: 13.24	F1: 18.50
	rouge-3:	P: 19.51	R:  8.25	F1: 11.59
	rouge-4:	P: 11.25	R:  4.69	F1:  6.62
	rouge-l:	P: 39.91	R: 20.40	F1: 26.99
	rouge-w:	P: 24.88	R:  4.42	F1:  7.50

Evaluation with Best
	rouge-1:	P: 60.61	R: 27.09	F1: 37.43
	rouge-2:	P: 30.70	R: 13.24	F1: 18.50
	rouge-3:	P: 19.51	R:  8.25	F1: 11.59
	rouge-4:	P: 11.25	R:  4.69	F1:  6.62
	rouge-l:	P: 39.91	R: 20.40	F1: 26.99
	rouge-w:	P: 24.88	R:  4.42	F1:  7.50



In [ ]:
print(gen_summary[1])

The Pentagon broke federal law by exchanging five Taliban detainees for Army Sgt. Susan Poling, general counsel for the U.S. Government Accountability Office, wrote in a letter to nine Republican senators that the Pentagon should have notified "the relevant congressional committees at least 30 days in advance of the transfer.. Bowe Bergdahl without giving Congress the appropriate notice, Congress' independent investigative arm announced Thursday. Moreover, Poling noted that the GAO also concluded the Defense Department broke the law by using "appropriated funds to carry out the transfer when no money was available for that purpose.. Poling noted that her office was not weighing in on Defense Secretary Chuck Hagel's decision to carry out the prisoner swap, but rather his "responsibility to comply with a notification condition on the availability of appropriations to transfer individuals from Guantanamo Bay.. Talking to CNN on Thursday evening, Pentagon spokesman Rear Adm. John Kirby not

In [ ]:
gen_summary = []
for i in range(0,4):
  try:
    file = open('/content/drive/My Drive/Colab Notebooks/pickle/gen_summaries_cnndm_{}.pkl'.format(i),'rb+')
    sublist = pickle.load(file)
    print(len(sublist))
    file.close()
    for s in sublist:
      gen_summary.append(s)
  except Exception as e:
    print(i)
    print(e)

100
100
100
100


In [ ]:
print(len(gen_summary))
print(gen_summary[120])

622



In [ ]:
gen_summary.pop(120)
summary.pop(120)
gen_summary.pop(457)
summary.pop(457)
gen_summary.pop(490)
summary.pop(490)
gen_summary.pop(549)
summary.pop(549)
gen_summary.pop(602)
summary.pop(602)
print(len(gen_summary))
print(len(summary))

617
617


In [ ]:
for i in range(len(summary)):
  file = open('/content/drive/My Drive/Colab Notebooks/summaries/{}_summary.txt'.format(i),'w')
  file.write(summary[i])
  file.close()
  file = open('/content/drive/My Drive/Colab Notebooks/gen_summaries/{}_summary.txt'.format(i),'w')
  file.write(gen_summary[i])
  file.close()
  

In [ ]:
!pip install git+https://github.com/tagucci/pythonrouge.git

  Cloning https://github.com/tagucci/pythonrouge.git to /tmp/pip-req-build-aqn08_cx
  Running command git clone -q https://github.com/tagucci/pythonrouge.git /tmp/pip-req-build-aqn08_cx
  Created wheel for pythonrouge: filename=pythonrouge-0.2-cp36-none-any.whl size=285413 sha256=aa23e6bdc3db32f174a1e14ccbe9873584a85ca71b5e8b05e72888c5a375d646
  Stored in directory: /tmp/pip-ephem-wheel-cache-xl4euysm/wheels/fd/ff/be/6716935d513fa8656ab185cb0aa70aed382b72dda42bf09c95
Successfully built pythonrouge


bin	     LICENSE.txt  pyrouge	    README.rst	setup.py
CHANGES.txt  PKG-INFO	  pyrouge.egg-info  setup.cfg


In [ ]:
from pythonrouge.pythonrouge import Pythonrouge

# system summary(predict) & reference summary

# initialize setting of ROUGE to eval ROUGE-1, 2, SU4
# if you evaluate ROUGE by sentence list as above, set summary_file_exist=False
# if recall_only=True, you can get recall scores of ROUGE
rouge = Pythonrouge(summary_file_exist=False,
                    summary=gen_summary, reference=summary,
                    n_gram=2, ROUGE_SU4=True, ROUGE_L=True,
                    recall_only=False, stemming=True, stopwords=True,
                    word_level=True, length_limit=False, length=50,
                    use_cf=False, cf=95, scoring_formula='average',
                    resampling=True, samples=617, favor=True, p=0.5)
rouge.ROUGE_path='/content/drive/My Drive/Colab Notebooks/rouge/RELEASE-1.5.5/ROUGE-1.5.5.pl'

rouge.data_path='/content/drive/My Drive/Colab Notebooks/rouge/RELEASE-1.5.5/data/'
score = rouge.calc_score()
print(score)

CalledProcessError: ignored

In [19]:
def sort_by_values_len(dict):
  dict_len= {key: len(value) for key, value in dict.items()}
  import operator
  sorted_key_list = sorted(dict_len.items(), key=operator.itemgetter(1), reverse=True)
  sorted_dict = [{item[0]: dict[item [0]]} for item in sorted_key_list]
  return sorted_dict

In [20]:
def dominating_set(st, sentences, graph):
  dom_set = []
  for token in st:
    dom_set.extend(graph[token])
  if(dom_set.sort() == sentences.sort()):
    return True
  else:
    return False

In [32]:
gen_summary_ds = []
for it in range(126, 127):
  graph = {}
  print('\r', it, end = '')
  sentences = text[it]
  for sentence1 in sentences:
   adj_list = []
   for sentence2 in sentences:
     if sentence1 == sentence2:
       continue
     s = []
     s.append(sentence1)
     s.append(sentence2)
     if len(s) != 2:
       continue
     try:
       with torch.no_grad():
         prediction = predict(s[0], s[1])
         _, prediction = prediction.max(1)
         #print(prediction)
         if prediction == 2:
          #print("yes")
          adj_list.append(sentence2)
     except Exception as e:
       print(e)
     if len(adj_list) > 0:
       graph[sentence1] = adj_list
  graph_list = sort_by_values_len(graph)
  result = []
  if len(graph)>0 :
    h = 0
    result.append(list(graph_list[h].keys())[0])
    while(dominating_set(result, sentences, graph) == 0):
      h = h + 1
      result.append(list(graph_list[h].keys())[0])
  else:
    print(" discard {} \n".format(it))
  result = [x[:len(x)-1] + "." for x in result]
  result = "".join(result)
  print(result)
  gen_summary_ds.append(result)
#file = open('/content/drive/My Drive/Colab Notebooks/pickle/gen_summaries_cnndm_ds_0.pkl','wb+')
#pickle.dump((gen_summary_ds), file)
#file.close()

 126

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


"When Congress comes back they will have to kind of rethink the plan and rethink particularly how they present it to the public.


In [ ]:
gen_summary_ds = []
for it in range(100, 200):
  graph = {}
  print('\r', it, end = '')
  sentences = text[it]
  for sentence1 in sentences:
   adj_list = []
   for sentence2 in sentences:
     if sentence1 == sentence2:
       continue
     s = []
     s.append(sentence1)
     s.append(sentence2)
     if len(s) != 2:
       continue
     try:
       with torch.no_grad():
         prediction = predict(s[0], s[1])
         _, prediction = prediction.max(1)
         #print(prediction)
         if prediction == 2:
          #print("yes")
          adj_list.append(sentence2)
     except Exception as e:
       print(e)
     if len(adj_list) > 0:
       graph[sentence1] = adj_list
  graph_list = sort_by_values_len(graph)
  result = []
  if len(graph)>0 :
    h = 0
    result.append(list(graph_list[h].keys())[0])
    while(dominating_set(result, sentences, graph) == 0):
      h = h + 1
      result.append(list(graph_list[h].keys())[0])
  else:
    print(" discard {} \n".format(it))
  result = [x[:len(x)-1] + "." for x in result]
  result = "".join(result)
  gen_summary_ds.append(result)
file = open('/content/drive/My Drive/Colab Notebooks/pickle/gen_summaries_cnndm_ds_1.pkl','wb+')
pickle.dump((gen_summary), file)
file.close()

 100

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


 120Trying to create tensor with negative dimension -227: [-227, 768]
Trying to create tensor with negative dimension -227: [-227, 768]
Trying to create tensor with negative dimension -227: [-227, 768]
Trying to create tensor with negative dimension -227: [-227, 768]
 discard 120 

 121 discard 121 

 122 discard 122 

 136 discard 136 

 139 discard 139 

 149 discard 149 

 158 discard 158 

 168 discard 168 

 181 discard 181 

 198 discard 198 

 199

NameError: ignored

In [ ]:
file = open('/content/drive/My Drive/Colab Notebooks/pickle/gen_summaries_cnndm_ds_1.pkl','wb+')
pickle.dump((gen_summary_ds), file)
file.close()

In [ ]:
gen_summary_ds = []
for it in range(200, 300):
  graph = {}
  print('\r', it, end = '')
  sentences = text[it]
  for sentence1 in sentences:
   adj_list = []
   for sentence2 in sentences:
     if sentence1 == sentence2:
       continue
     s = []
     s.append(sentence1)
     s.append(sentence2)
     if len(s) != 2:
       continue
     try:
       with torch.no_grad():
         prediction = predict(s[0], s[1])
         _, prediction = prediction.max(1)
         #print(prediction)
         if prediction == 2:
          #print("yes")
          adj_list.append(sentence2)
     except Exception as e:
       print(e)
     if len(adj_list) > 0:
       graph[sentence1] = adj_list
  graph_list = sort_by_values_len(graph)
  result = []
  if len(graph)>0 :
    h = 0
    result.append(list(graph_list[h].keys())[0])
    while(dominating_set(result, sentences, graph) == 0):
      h = h + 1
      result.append(list(graph_list[h].keys())[0])
  else:
    print(" discard {} \n".format(it))
  result = [x[:len(x)-1] + "." for x in result]
  result = "".join(result)
  gen_summary_ds.append(result)
file = open('/content/drive/My Drive/Colab Notebooks/pickle/gen_summaries_cnndm_ds_2.pkl','wb+')
pickle.dump((gen_summary_ds), file)
file.close()



 200

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


 discard 200 

 202 discard 202 

 206 discard 206 

 217 discard 217 

 240 discard 240 

 253 discard 253 

 260 discard 260 

 270 discard 270 

 297 discard 297 

 299

In [ ]:
gen_summary_ds = []
for it in range(300, 400):
  graph = {}
  print('\r', it, end = '')
  sentences = text[it]
  for sentence1 in sentences:
   adj_list = []
   for sentence2 in sentences:
     if sentence1 == sentence2:
       continue
     s = []
     s.append(sentence1)
     s.append(sentence2)
     if len(s) != 2:
       continue
     try:
       with torch.no_grad():
         prediction = predict(s[0], s[1])
         _, prediction = prediction.max(1)
         #print(prediction)
         if prediction == 2:
          #print("yes")
          adj_list.append(sentence2)
     except Exception as e:
       print(e)
     if len(adj_list) > 0:
       graph[sentence1] = adj_list
  graph_list = sort_by_values_len(graph)
  result = []
  if len(graph)>0 :
    h = 0
    result.append(list(graph_list[h].keys())[0])
    while(dominating_set(result, sentences, graph) == 0):
      h = h + 1
      result.append(list(graph_list[h].keys())[0])
  else:
    print(" discard {} \n".format(it))
  result = [x[:len(x)-1] + "." for x in result]
  result = "".join(result)
  gen_summary_ds.append(result)
file = open('/content/drive/My Drive/Colab Notebooks/pickle/gen_summaries_cnndm_ds_3.pkl','wb+')
pickle.dump((gen_summary_ds), file)
file.close()



 300

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


 301 discard 301 

 316 discard 316 

 319 discard 319 

 320 discard 320 

 343 discard 343 

 349 discard 349 

 351 discard 351 

 352 discard 352 

 355 discard 355 

 358 discard 358 

 367 discard 367 

 380 discard 380 

 386 discard 386 

 391 discard 391 

 392 discard 392 

 396 discard 396 

 399

In [ ]:
gen_summary_ds = []
for it in range(400, 500):
  graph = {}
  print('\r', it, end = '')
  sentences = text[it]
  for sentence1 in sentences:
   adj_list = []
   for sentence2 in sentences:
     if sentence1 == sentence2:
       continue
     s = []
     s.append(sentence1)
     s.append(sentence2)
     if len(s) != 2:
       continue
     try:
       with torch.no_grad():
         prediction = predict(s[0], s[1])
         _, prediction = prediction.max(1)
         #print(prediction)
         if prediction == 2:
          #print("yes")
          adj_list.append(sentence2)
     except Exception as e:
       print(e)
     if len(adj_list) > 0:
       graph[sentence1] = adj_list
  graph_list = sort_by_values_len(graph)
  result = []
  if len(graph)>0 :
    h = 0
    result.append(list(graph_list[h].keys())[0])
    while(dominating_set(result, sentences, graph) == 0):
      h = h + 1
      result.append(list(graph_list[h].keys())[0])
  else:
    print(" discard {} \n".format(it))
  result = [x[:len(x)-1] + "." for x in result]
  result = "".join(result)
  gen_summary_ds.append(result)
file = open('/content/drive/My Drive/Colab Notebooks/pickle/gen_summaries_cnndm_ds_4.pkl','wb+')
pickle.dump((gen_summary_ds), file)
file.close()



 400

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


 401 discard 401 

 424 discard 424 

 428 discard 428 

 429 discard 429 

 449 discard 449 

 450 discard 450 

 457Trying to create tensor with negative dimension -22: [-22, 768]
Trying to create tensor with negative dimension -22: [-22, 768]
Trying to create tensor with negative dimension -22: [-22, 768]
Trying to create tensor with negative dimension -22: [-22, 768]
Trying to create tensor with negative dimension -22: [-22, 768]
Trying to create tensor with negative dimension -22: [-22, 768]
Trying to create tensor with negative dimension -22: [-22, 768]
Trying to create tensor with negative dimension -22: [-22, 768]
Trying to create tensor with negative dimension -22: [-22, 768]
Trying to create tensor with negative dimension -22: [-22, 768]
Trying to create tensor with negative dimension -22: [-22, 768]
Trying to create tensor with negative dimension -22: [-22, 768]
Trying to create tensor with negative dimension -22: [-22, 768]
Trying to create tensor with negative dimension -2

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (772 > 512). Running this sequence through BERT will result in indexing errors


 490

...

Trying to create tensor with negative dimension -522: [-522, 768]


Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (772 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (772 > 512). Running this sequence through BERT will result in indexing errors


Trying to create tensor with negative dimension -522: [-522, 768]
Trying to create tensor with negative dimension -522: [-522, 768]


Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (772 > 512). Running this sequence through BERT will result in indexing errors


Trying to create tensor with negative dimension -522: [-522, 768]
Trying to create tensor with negative dimension -522: [-522, 768]
 discard 490 

 494 discard 494 

 495 discard 495 

 499

In [ ]:
gen_summary_ds = []
for it in range(500, 600):
  graph = {}
  print('\r', it, end = '')
  sentences = text[it]
  for sentence1 in sentences:
   adj_list = []
   for sentence2 in sentences:
     if sentence1 == sentence2:
       continue
     s = []
     s.append(sentence1)
     s.append(sentence2)
     if len(s) != 2:
       continue
     try:
       with torch.no_grad():
         prediction = predict(s[0], s[1])
         _, prediction = prediction.max(1)
         #print(prediction)
         if prediction == 2:
          #print("yes")
          adj_list.append(sentence2)
     except Exception as e:
       print(e)
     if len(adj_list) > 0:
       graph[sentence1] = adj_list
  graph_list = sort_by_values_len(graph)
  result = []
  if len(graph)>0 :
    h = 0
    result.append(list(graph_list[h].keys())[0])
    while(dominating_set(result, sentences, graph) == 0):
      h = h + 1
      result.append(list(graph_list[h].keys())[0])
  else:
    print(" discard {} \n".format(it))
  result = [x[:len(x)-1] + "." for x in result]
  result = "".join(result)
  gen_summary_ds.append(result)
file = open('/content/drive/My Drive/Colab Notebooks/pickle/gen_summaries_cnndm_ds_5.pkl','wb+')
pickle.dump((gen_summary_ds), file)
file.close()



 500

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


 518 discard 518 

 520 discard 520 

 522 discard 522 

 524 discard 524 

 525 discard 525 

 534 discard 534 

 549Trying to create tensor with negative dimension -44: [-44, 768]
Trying to create tensor with negative dimension -44: [-44, 768]
Trying to create tensor with negative dimension -44: [-44, 768]
Trying to create tensor with negative dimension -44: [-44, 768]
Trying to create tensor with negative dimension -44: [-44, 768]
Trying to create tensor with negative dimension -44: [-44, 768]
Trying to create tensor with negative dimension -44: [-44, 768]
Trying to create tensor with negative dimension -44: [-44, 768]
Trying to create tensor with negative dimension -44: [-44, 768]
Trying to create tensor with negative dimension -44: [-44, 768]
 556 discard 556 

 560 discard 560 

 566 discard 566 

 569 discard 569 

 576 discard 576 

 582 discard 582 

 584 discard 584 

 590 discard 590 

 599

In [ ]:
gen_summary_ds = []
for it in range(600, 622):
  graph = {}
  print('\r', it, end = '')
  sentences = text[it]
  for sentence1 in sentences:
   adj_list = []
   for sentence2 in sentences:
     if sentence1 == sentence2:
       continue
     s = []
     s.append(sentence1)
     s.append(sentence2)
     if len(s) != 2:
       continue
     try:
       with torch.no_grad():
         prediction = predict(s[0], s[1])
         _, prediction = prediction.max(1)
         #print(prediction)
         if prediction == 2:
          #print("yes")
          adj_list.append(sentence2)
     except Exception as e:
       print(e)
     if len(adj_list) > 0:
       graph[sentence1] = adj_list
  graph_list = sort_by_values_len(graph)
  result = []
  if len(graph)>0 :
    h = 0
    result.append(list(graph_list[h].keys())[0])
    while(dominating_set(result, sentences, graph) == 0):
      h = h + 1
      result.append(list(graph_list[h].keys())[0])
  else:
    print(" discard {} \n".format(it))
  result = [x[:len(x)-1] + "." for x in result]
  result = "".join(result)
  gen_summary_ds.append(result)
file = open('/content/drive/My Drive/Colab Notebooks/pickle/gen_summaries_cnndm_ds_6.pkl','wb+')
pickle.dump((gen_summary_ds), file)
file.close()

 600

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


 601 discard 601 

 602Trying to create tensor with negative dimension -24: [-24, 768]
Trying to create tensor with negative dimension -24: [-24, 768]
Trying to create tensor with negative dimension -24: [-24, 768]
Trying to create tensor with negative dimension -24: [-24, 768]
Trying to create tensor with negative dimension -24: [-24, 768]
Trying to create tensor with negative dimension -24: [-24, 768]
Trying to create tensor with negative dimension -24: [-24, 768]
Trying to create tensor with negative dimension -24: [-24, 768]
Trying to create tensor with negative dimension -24: [-24, 768]
Trying to create tensor with negative dimension -24: [-24, 768]
Trying to create tensor with negative dimension -24: [-24, 768]
Trying to create tensor with negative dimension -24: [-24, 768]
Trying to create tensor with negative dimension -24: [-24, 768]
Trying to create tensor with negative dimension -24: [-24, 768]
Trying to create tensor with negative dimension -24: [-24, 768]
Trying to create 

In [ ]:
gen_summary_ds = []
for i in range(0,7):
  try:
    file = open('/content/drive/My Drive/Colab Notebooks/pickle/gen_summaries_cnndm_ds_{}.pkl'.format(i),'rb+')
    sublist = pickle.load(file)
    print(len(sublist))
    file.close()
    for s in sublist:
      gen_summary_ds.append(s)
  except Exception as e:
    print(i)
    print(e)

100
100
100
100
100
100
22


In [ ]:
gen_summary_ds.pop(619)
summary.pop(619)
gen_summary_ds.pop(615)
summary.pop(615)
gen_summary_ds.pop(612)
summary.pop(612)
gen_summary_ds.pop(601)
summary.pop(601)
gen_summary_ds.pop(590)
summary.pop(590)
gen_summary_ds.pop(584)
summary.pop(584)
gen_summary_ds.pop(582)
summary.pop(582)
gen_summary_ds.pop(576)
summary.pop(576)
gen_summary_ds.pop(569)
summary.pop(569)
gen_summary_ds.pop(566)
summary.pop(566)
gen_summary_ds.pop(560)
summary.pop(560)
gen_summary_ds.pop(556)
summary.pop(556)
gen_summary_ds.pop(534)
summary.pop(534)
gen_summary_ds.pop(525)
summary.pop(525)
gen_summary_ds.pop(524)
summary.pop(524)
gen_summary_ds.pop(522)
summary.pop(520)
gen_summary_ds.pop(518)
summary.pop(518)
gen_summary_ds.pop(495)
summary.pop(495)
gen_summary_ds.pop(494)
summary.pop(494)
gen_summary_ds.pop(490)
summary.pop(490)
gen_summary_ds.pop(481)
summary.pop(481)
gen_summary_ds.pop(480)
summary.pop(480)
gen_summary_ds.pop(469)
summary.pop(469)
gen_summary_ds.pop(466)
summary.pop(466)
gen_summary_ds.pop(457)
summary.pop(457)
gen_summary_ds.pop(450)
summary.pop(450)
gen_summary_ds.pop(449)
summary.pop(449)
gen_summary_ds.pop(429)
summary.pop(429)
gen_summary_ds.pop(428)
summary.pop(428)
gen_summary_ds.pop(424)
summary.pop(424)
gen_summary_ds.pop(401)
summary.pop(401)
gen_summary_ds.pop(396)
summary.pop(396)
gen_summary_ds.pop(392)
summary.pop(392)
gen_summary_ds.pop(391)
summary.pop(391)
gen_summary_ds.pop(386)
summary.pop(386)
gen_summary_ds.pop(380)
summary.pop(380)
gen_summary_ds.pop(367)
summary.pop(367)
gen_summary_ds.pop(358)
summary.pop(358)
gen_summary_ds.pop(355)
summary.pop(355)
gen_summary_ds.pop(352)
summary.pop(352)
gen_summary_ds.pop(351)
summary.pop(351)
gen_summary_ds.pop(349)
summary.pop(349)
gen_summary_ds.pop(343)
summary.pop(343)
gen_summary_ds.pop(320)
summary.pop(320)
gen_summary_ds.pop(319)
summary.pop(319)
gen_summary_ds.pop(316)
summary.pop(316)
gen_summary_ds.pop(301)
summary.pop(301)
gen_summary_ds.pop(297)
summary.pop(297)
gen_summary_ds.pop(270)
summary.pop(270)
gen_summary_ds.pop(260)
summary.pop(260)
gen_summary_ds.pop(253)
summary.pop(253)
gen_summary_ds.pop(240)
summary.pop(240)
gen_summary_ds.pop(217)
summary.pop(217)
gen_summary_ds.pop(206)
summary.pop(206)
gen_summary_ds.pop(202)
summary.pop(202)
gen_summary_ds.pop(200)
summary.pop(200)
gen_summary_ds.pop(198)
summary.pop(198)
gen_summary_ds.pop(181)
summary.pop(181)
gen_summary_ds.pop(168)
summary.pop(168)
gen_summary_ds.pop(158)
summary.pop(158)
gen_summary_ds.pop(149)
summary.pop(149)
gen_summary_ds.pop(139)
summary.pop(139)
gen_summary_ds.pop(136)
summary.pop(136)
gen_summary_ds.pop(122)
summary.pop(122)
gen_summary_ds.pop(121)
summary.pop(121)
gen_summary_ds.pop(120)
summary.pop(120)
gen_summary_ds.pop(94)
summary.pop(94)
gen_summary_ds.pop(92)
summary.pop(92)
gen_summary_ds.pop(91)
summary.pop(91)
gen_summary_ds.pop(85)
summary.pop(85)
gen_summary_ds.pop(74)
summary.pop(74)
gen_summary_ds.pop(60)
summary.pop(60)
gen_summary_ds.pop(56)
summary.pop(56)
gen_summary_ds.pop(6)
summary.pop(6)
gen_summary_ds.pop(2)
summary.pop(2)


'The Monty Python team will be doing their first live show since 1980. It\'s been years since "Monty Python\'s Flying Circus" debuted. Will their sketch comedy still work? Their fan base is solid.'

In [ ]:
print(len(summary))
print(len(gen_summary_ds))

622
622


In [ ]:
!pip install py-rouge
import rouge

def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

for aggregator in ['Avg', 'Best']:
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)


    
    all_hypothesis = summary
    all_references = gen_summary_ds

    scores = evaluator.get_scores(all_hypothesis, all_references)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
            for hypothesis_id, results_per_ref in enumerate(results):
                nb_references = len(results_per_ref['p'])
                for reference_id in range(nb_references):
                    print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                    print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
            print()
        else:
            print(prepare_results(results['p'], results['r'], results['f']))
    print()

Evaluation with Avg
	rouge-1:	P: 22.28	R: 33.31	F1: 24.80
	rouge-2:	P:  6.77	R: 10.41	F1:  7.66
	rouge-3:	P:  3.34	R:  5.41	F1:  3.88
	rouge-4:	P:  2.12	R:  3.59	F1:  2.51
	rouge-l:	P: 19.97	R: 28.34	F1: 22.22
	rouge-w:	P: 12.26	R:  9.75	F1:  9.91

Evaluation with Best


ZeroDivisionError: ignored

In [ ]:
!pip install py-rouge
import rouge

def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

for aggregator in ['Avg', 'Best']:
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)


    
    all_hypothesis = summary
    all_references = gen_summary_ds

    scores = evaluator.get_scores(all_hypothesis, all_references)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
            for hypothesis_id, results_per_ref in enumerate(results):
                nb_references = len(results_per_ref['p'])
                for reference_id in range(nb_references):
                    print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                    print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
            print()
        else:
            print(prepare_results(results['p'], results['r'], results['f']))
    print()

Evaluation with Avg
	rouge-1:	P: 19.62	R: 29.32	F1: 21.83
	rouge-2:	P:  5.96	R:  9.16	F1:  6.74
	rouge-3:	P:  2.93	R:  4.76	F1:  3.41
	rouge-4:	P:  1.86	R:  3.15	F1:  2.21
	rouge-l:	P: 17.58	R: 24.94	F1: 19.56
	rouge-w:	P: 10.80	R:  8.58	F1:  8.72

Evaluation with Best


ZeroDivisionError: ignored

In [ ]:
gen_summary = []
for i in range(0,6):
  try:
    file = open('/content/drive/My Drive/Colab Notebooks/pickle/gen_summaries_cnndm_{}.pkl'.format(i),'rb+')
    sublist = pickle.load(file)
    print(len(sublist))
    file.close()
    for s in sublist:
      gen_summary.append(s)
  except Exception as e:
    print(i)
    print(e)

100
100
100
100
100
122


In [ ]:
print(len(summary))
print(len(gen_summary))

622
622


In [ ]:
!pip install py-rouge
import rouge

def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

for aggregator in ['Avg', 'Best']:
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)


    
    all_hypothesis = summary
    all_references = gen_summary

    scores = evaluator.get_scores(all_hypothesis, all_references)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
            for hypothesis_id, results_per_ref in enumerate(results):
                nb_references = len(results_per_ref['p'])
                for reference_id in range(nb_references):
                    print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                    print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
            print()
        else:
            print(prepare_results(results['p'], results['r'], results['f']))
    print()

Evaluation with Avg
	rouge-1:	P: 44.20	R: 22.12	F1: 28.49
	rouge-2:	P: 17.06	R:  8.36	F1: 10.86
	rouge-3:	P:  9.53	R:  4.65	F1:  6.04
	rouge-4:	P:  6.38	R:  3.08	F1:  4.01
	rouge-l:	P: 32.95	R: 18.13	F1: 22.80
	rouge-w:	P: 21.02	R:  4.10	F1:  6.68

Evaluation with Best


ZeroDivisionError: ignored

In [ ]:
gen_summary.pop(619)
summary.pop(619)
gen_summary.pop(615)
summary.pop(615)
gen_summary.pop(612)
summary.pop(612)
gen_summary.pop(601)
summary.pop(601)
gen_summary.pop(590)
summary.pop(590)
gen_summary.pop(584)
summary.pop(584)
gen_summary.pop(582)
summary.pop(582)
gen_summary.pop(576)
summary.pop(576)
gen_summary.pop(569)
summary.pop(569)
gen_summary.pop(566)
summary.pop(566)
gen_summary.pop(560)
summary.pop(560)
gen_summary.pop(556)
summary.pop(556)
gen_summary.pop(534)
summary.pop(534)
gen_summary.pop(525)
summary.pop(525)
gen_summary.pop(524)
summary.pop(524)
gen_summary.pop(522)
summary.pop(520)
gen_summary.pop(518)
summary.pop(518)
gen_summary.pop(495)
summary.pop(495)
gen_summary.pop(494)
summary.pop(494)
gen_summary.pop(490)
summary.pop(490)
gen_summary.pop(481)
summary.pop(481)
gen_summary.pop(480)
summary.pop(480)
gen_summary.pop(469)
summary.pop(469)
gen_summary.pop(466)
summary.pop(466)
gen_summary.pop(457)
summary.pop(457)
gen_summary.pop(450)
summary.pop(450)
gen_summary.pop(449)
summary.pop(449)
gen_summary.pop(429)
summary.pop(429)
gen_summary.pop(428)
summary.pop(428)
gen_summary.pop(424)
summary.pop(424)
gen_summary.pop(401)
summary.pop(401)
gen_summary.pop(396)
summary.pop(396)
gen_summary.pop(392)
summary.pop(392)
gen_summary.pop(391)
summary.pop(391)
gen_summary.pop(386)
summary.pop(386)
gen_summary.pop(380)
summary.pop(380)
gen_summary.pop(367)
summary.pop(367)
gen_summary.pop(358)
summary.pop(358)
gen_summary.pop(355)
summary.pop(355)
gen_summary.pop(352)
summary.pop(352)
gen_summary.pop(351)
summary.pop(351)
gen_summary.pop(349)
summary.pop(349)
gen_summary.pop(343)
summary.pop(343)
gen_summary.pop(320)
summary.pop(320)
gen_summary.pop(319)
summary.pop(319)
gen_summary.pop(316)
summary.pop(316)
gen_summary.pop(301)
summary.pop(301)
gen_summary.pop(297)
summary.pop(297)
gen_summary.pop(270)
summary.pop(270)
gen_summary.pop(260)
summary.pop(260)
gen_summary.pop(253)
summary.pop(253)
gen_summary.pop(240)
summary.pop(240)
gen_summary.pop(217)
summary.pop(217)
gen_summary.pop(206)
summary.pop(206)
gen_summary.pop(202)
summary.pop(202)
gen_summary.pop(200)
summary.pop(200)
gen_summary.pop(198)
summary.pop(198)
gen_summary.pop(181)
summary.pop(181)
gen_summary.pop(168)
summary.pop(168)
gen_summary.pop(158)
summary.pop(158)
gen_summary.pop(149)
summary.pop(149)
gen_summary.pop(139)
summary.pop(139)
gen_summary.pop(136)
summary.pop(136)
gen_summary.pop(122)
summary.pop(122)
gen_summary.pop(121)
summary.pop(121)
gen_summary.pop(120)
summary.pop(120)
gen_summary.pop(94)
summary.pop(94)
gen_summary.pop(92)
summary.pop(92)
gen_summary.pop(91)
summary.pop(91)
gen_summary.pop(85)
summary.pop(85)
gen_summary.pop(74)
summary.pop(74)
gen_summary.pop(60)
summary.pop(60)
gen_summary.pop(56)
summary.pop(56)
gen_summary.pop(6)
summary.pop(6)
gen_summary.pop(2)
summary.pop(2)


'The Monty Python team will be doing their first live show since 1980. It\'s been years since "Monty Python\'s Flying Circus" debuted. Will their sketch comedy still work? Their fan base is solid.'

In [ ]:
!pip install py-rouge
import rouge

def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

for aggregator in ['Avg', 'Best']:
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)


    
    all_hypothesis = summary
    all_references = gen_summary

    scores = evaluator.get_scores(all_hypothesis, all_references)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
            for hypothesis_id, results_per_ref in enumerate(results):
                nb_references = len(results_per_ref['p'])
                for reference_id in range(nb_references):
                    print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                    print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
            print()
        else:
            print(prepare_results(results['p'], results['r'], results['f']))
    print()

Evaluation with Avg
	rouge-1:	P: 49.36	R: 24.55	F1: 31.70
	rouge-2:	P: 19.13	R:  9.33	F1: 12.14
	rouge-3:	P: 10.68	R:  5.17	F1:  6.74
	rouge-4:	P:  7.15	R:  3.43	F1:  4.47
	rouge-l:	P: 36.83	R: 20.17	F1: 25.42
	rouge-w:	P: 23.51	R:  4.55	F1:  7.43

Evaluation with Best


ZeroDivisionError: ignored